# RNNを試してみる

### 必要なパッケージのインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)

Using TensorFlow backend.


### 実験用データの生成

In [2]:
def sin(x, T=100):
    return np.sin(2.0 * np.pi * x / T)


def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2 * T + 1)
    noise = ampl * np.random.uniform(low=-1.0, high=1.0, size=len(x))
    return sin(x) + noise

T = 100
f = toy_problem(T)

length_of_sequences = 2 * T
maxlen = 25  # ひとつの時系列データの長さ

data = []
target = []

for i in range(0, length_of_sequences - maxlen + 1):
    data.append(f[i: i + maxlen])
    target.append(f[i + maxlen])

X = np.array(data).reshape(len(data), maxlen, 1)
Y = np.array(target).reshape(len(data), 1)

# データ設定
N_train = int(len(data) * 0.9)
N_validation = len(data) - N_train

X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X, Y, test_size=N_validation)

### モデルの定義

In [7]:
n_in = len(X[0][0])  # 1
n_hidden = 20
n_out = len(Y[0])  # 1


def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)


early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model = Sequential()
model.add(SimpleRNN(n_hidden,
                    kernel_initializer=weight_variable,
                    input_shape=(maxlen, n_in)))
model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('linear'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error',
              optimizer=optimizer)

### モデルの学習

In [8]:
epochs = 500
batch_size = 10

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, Y_validation),
          callbacks=[early_stopping])

Train on 158 samples, validate on 18 samples
Epoch 1/500
158/158 [==============================] - 0s - loss: 0.5048 - val_loss: 0.3826
Epoch 2/500
158/158 [==============================] - 0s - loss: 0.3965 - val_loss: 0.2554
Epoch 3/500
158/158 [==============================] - 0s - loss: 0.2214 - val_loss: 0.0978
Epoch 4/500
158/158 [==============================] - 0s - loss: 0.0854 - val_loss: 0.0594
Epoch 5/500
158/158 [==============================] - 0s - loss: 0.0605 - val_loss: 0.0805
Epoch 6/500
158/158 [==============================] - 0s - loss: 0.0445 - val_loss: 0.0364
Epoch 7/500
158/158 [==============================] - 0s - loss: 0.0394 - val_loss: 0.0335
Epoch 8/500
158/158 [==============================] - 0s - loss: 0.0388 - val_loss: 0.0293
Epoch 9/500
158/158 [==============================] - 0s - loss: 0.0311 - val_loss: 0.0340
Epoch 10/500
158/158 [==============================] - 0s - loss: 0.0258 - val_loss: 0.0376
Epoch 11/500
158/158 [============